In [ ]:
# Import the required libraries

import pymongo
import datetime
import collections

import pandas as pd
import scipy.stats

In [ ]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels

In [ ]:
# Load the expanded names of keys and human-readable codes into memory
# Don't bother reading this cell yet: we'll use it in a bit.

expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

## Activity 1
How many accidents were there at less than 30mph?

Create a DataFrame that holds the accident index, number of vehicles, and number of casualties for each accident, where the speed limit is less than 30mph.

In [ ]:
accidents.find({'Speed_limit': {'$lt': 30}}).count()

In [ ]:
pd.DataFrame(list(accidents.find({'Speed_limit': {'$lt': 30}}, 
                                 ['Accident_Index', 'Number_of_Vehicles', 'Number_of_Casualties'])))

### Activity 2
Find and display the document in the `labels` collection that stores details of weather conditions. 

Use the `label_of` dict to find all the labels for `Weather_Conditions`.

Use the `label_of` dict in a list comprehension to find all the labels for `Weather_Conditions` that refer to wind. Note that you can include more than one `if` clause in a list comprehension: just put them one after the other, like this:

```
[item for item in thing if condition1 if condition2]
```
    
It will only return items where both conditions are true.

In [ ]:
labels.find_one({'label': 'Weather_Conditions'})

In [ ]:
[label_of[key, code] for (key, code) in label_of if key == 'Weather_Conditions']

In [ ]:
[label_of[key, code] for (key, code) in label_of 
 if key == 'Weather_Conditions' 
 if 'wind' in label_of[key, code]]

### Activity 3
The earlier query is below, but it prints the results rather than putting them in a DataFrame.

Repeat this query, but print the labels of the accident severity, road type, and weather conditions (e.g. 
```
201201BS70001 Slight Single carriageway Fine no high winds
```
)

In [ ]:
for a in accidents.find({'Speed_limit': 30, 'Number_of_Casualties': 1, 'Number_of_Vehicles': 2}, limit=10):
    print(a['Accident_Index'],  
          label_of['Accident_Severity', a['Accident_Severity']],
          label_of['Road_Type', a['Road_Type']], 
          label_of['Weather_Conditions', a['Weather_Conditions']])

### Activity 4
What are the numbers of accidents at each severity in 60mph zones? Show your results as a bar chart.

In [ ]:
# Build a DataFrame, one row for each accident
severities_60_unrolled_df = pd.DataFrame(list(accidents.find({'Speed_limit': 60}, ['Accident_Severity'])))

# Count the number of each severity
severities_60_ss = severities_60_unrolled_df['Accident_Severity'].value_counts()
severities_60_ss.sort_index(inplace=True)
severities_60_ss.index = [label_of['Accident_Severity', r] for r in severities_60_ss.index]
severities_60_ss.plot(kind='bar')

### Activity 5
What are the proportions of accident severities by junction type (roundabout, crossroads, etc)?

This is a more in-depth activity than before, so I've broken it down into steps.

* What are the different types of junction? You'll need to look at the `label_of` dict to find the right key
* What different severities happen at each junction type?

In [ ]:
# Find labels that refer to junctions
set(key for key, _ in label_of if 'Junction' in key)

In [ ]:
# All the junction-related labels.
list(sorted((key, label_of[key, code]) for key, code in label_of if 'Junction' in key))

"Junction_Detail" is the information we want.

We can now build a Counter that summarises the counts at each junction detail/severity combination.

In [ ]:
# Build a DataFrame, one row for each accident
severity_by_junction_unrolled_df = pd.DataFrame(list(accidents.find({}, ['Junction_Detail', 'Accident_Severity'])))

# Count the number of each severity
severity_by_junction_df = pd.crosstab(severity_by_junction_unrolled_df['Junction_Detail'], 
                                      severity_by_junction_unrolled_df['Accident_Severity'])
severity_by_junction_df

In [ ]:
# Relabel the index to the junction types
severity_by_junction_df.index = [label_of['Junction_Detail', code] for code in severity_by_junction_df.index]
severity_by_junction_df.index.name = 'Junction type'

# Relabel the columns
severity_by_junction_df.columns = [label_of['Accident_Severity', s] for s in severity_by_junction_df.columns]
severity_by_junction_df.columns.name = 'Accident severity'
severity_by_junction_df

In [ ]:
severity_by_junction_df.plot(kind='bar')

Again, it's difficult to judge if the proportions of accident severities are different for different junction types. We'll look at this later.

### Activity 6
We can also ask if different age people respond to injuries differently. In other words, are old people more frail and so therefore more likely to be seriously or fatally injured?

Following the examples above, let's build a DataFrame that counts the number of casualties for each age band / severity combination, and use the human-readable labels for the data rather than the codes.

You should end up with a `DataFrame` called `severity_by_age_df` that looks like this:


Severity | Fatal | Serious | Slight
-|-|-|-
Unknown | 0 | 0 | 8
0 - 5 | 0 | 3 | 20
6 - 10 | 0 | 1 | 28
11 - 15 | 0 | 3 | 34
16 - 20 | 1 | 11 | 115
21 - 25 | 1 | 16 | 133
26 - 35 | 2 | 15 | 219
36 - 45 | 3 | 12 | 168
46 - 55 | 0 | 10 | 125
56 - 65 | 1 | 7 | 63
66 - 75 | 0 | 9 | 28
Over 75 | 2 | 2 | 20

In [ ]:
# Build a DataFrame, one row for each accident
severity_by_age_unrolled_df = pd.DataFrame({'Age_Band_of_Casualty': c['Age_Band_of_Casualty'],
  'Casualty_Severity': c['Casualty_Severity']}
  for a in accidents.find({'Local_Authority_(Highway)': 'E06000042'})
 for c in a['Casualties'])

# Count the number of each severity
severity_by_age_df = pd.crosstab(severity_by_age_unrolled_df['Age_Band_of_Casualty'], 
                                      severity_by_age_unrolled_df['Casualty_Severity'])
severity_by_age_df

In [ ]:
# Relabel the index to the age bands
severity_by_age_df.index = [label_of['Age_Band_of_Casualty', code] for code in severity_by_age_df.index]
severity_by_age_df.index = ['Unknown'] + list(severity_by_age_df.index[1:])
severity_by_age_df.index.name = 'Age band'

# Relabel the columns
severity_by_age_df.columns = [label_of['Casualty_Severity', s] for s in severity_by_age_df.columns]
severity_by_age_df.columns.name = 'Casualty severity'
severity_by_age_df

### Activity 7

How does the number of vehicles in an accident vary with the speed limit?

Find the results, find the line of best fit, then plot the data and line of best fit on a scatter graph.

In [ ]:
# Build a DataFrame, one row for each accident
speed_veh_unrolled_df = pd.DataFrame(list(accidents.find({}, ['Speed_limit', 'Number_of_Vehicles'])))

# Count the number of each severity
speed_veh_df = pd.crosstab(speed_veh_unrolled_df['Speed_limit'], 
                                      speed_veh_unrolled_df['Number_of_Vehicles'])
speed_veh_df

In [ ]:
speed_veh_long_df = speed_veh_df.stack().reset_index()
speed_veh_long_df

In [ ]:
regressionline = scipy.stats.linregress(speed_veh_unrolled_df['Speed_limit'],
                                       speed_veh_unrolled_df['Number_of_Vehicles'])
# The regression line is of the form y = m x + b
m = regressionline[0]
b = regressionline[1]
(m, b)

In [ ]:
plt.scatter(speed_veh_long_df['Speed_limit'], 
            speed_veh_long_df['Number_of_Vehicles'],
            s=np.sqrt(speed_veh_long_df[0])*1.5,
            alpha=0.5
            )
plt.xlabel('Speed limit')
plt.ylabel('Number of vehicles')

x = np.linspace(10, 70, 20)
plt.plot(x, m*x + b)

plt.show()

In [ ]:
small_svcl_df = speed_veh_long_df[(speed_veh_long_df['Speed_limit'] >= 30) & 
                                  (speed_veh_long_df['Number_of_Vehicles'] <= 20)]

plt.scatter(small_svcl_df['Speed_limit'], 
            small_svcl_df['Number_of_Vehicles'],
            s=np.sqrt(small_svcl_df[0])*3,
            alpha=0.5
            )
plt.xlabel('Speed limit')
plt.ylabel('Number of vehicles')

x = np.linspace(30, 70, 20)
plt.plot(x, m*x + b)

plt.show()

The flat regression line indicates that there's no real correlation between speed limit and number of vehicles involved in an accident.